# Primitives examples

<LegacyContent>
<Admonition type="note">
This documentation is relevant to IBM Quantum&reg; Platform Classic. If you need the newer version, go to the new [IBM Quantum Platform documentation.](https://quantum.cloud.ibm.com/docs/guides/primitives-examples)
</Admonition>
</LegacyContent>
<CloudContent>
<Admonition type="note">
This documentation is relevant to the new IBM Quantum&reg; Platform. If you need the previous version, return to the [IBM Quantum Platform Classic documentation.](https://docs.quantum.ibm.com/guides/primitives-examples)
</Admonition>
</CloudContent>

The examples in this section illustrate some common ways to use primitives. Before running these examples, follow the instructions in [Install and set up.](install-qiskit)

<Admonition type="note">
    These examples all use the primitives from Qiskit Runtime, but you could use the base primitives instead.
</Admonition>

## Estimator examples

Efficiently calculate and interpret expectation values of the quantum operators required for many algorithms with Estimator. Explore uses in molecular modeling, machine learning, and complex optimization problems.

### Run a single experiment

Use Estimator to determine the expectation value of a single circuit-observable pair.

In [1]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
observable = SparsePauliOp("Z" * 50)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)

estimator = Estimator(mode=backend)
job = estimator.run([(isa_circuit, isa_observable)])
result = job.result()

print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")

 > Expectation value: -0.7916666666666666
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


### Run multiple experiments in a single job

Use Estimator to determine the expectation values of multiple circuit-observable pairs.

In [2]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]

pubs = []
circuits = [iqp(mat) for mat in mats]
observables = [
    SparsePauliOp("X" * 50),
    SparsePauliOp("Y" * 50),
    SparsePauliOp("Z" * 50),
]

# Get ISA circuits
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

for qc, obs in zip(circuits, observables):
    isa_circuit = pm.run(qc)
    isa_obs = obs.apply_layout(isa_circuit.layout)
    pubs.append((isa_circuit, isa_obs))

estimator = Estimator(backend)
job = estimator.run(pubs)
job_result = job.result()

for idx in range(len(pubs)):
    pub_result = job_result[idx]
    print(f">>> Expectation values for PUB {idx}: {pub_result.data.evs}")
    print(f">>> Standard errors for PUB {idx}: {pub_result.data.stds}")

>>> Expectation values for PUB 0: -0.08808290155440414
>>> Standard errors for PUB 0: 0.07584969096715767
>>> Expectation values for PUB 1: -0.07734806629834254
>>> Standard errors for PUB 1: 0.0941405420654118
>>> Expectation values for PUB 2: -0.03985507246376811
>>> Standard errors for PUB 2: 0.11134528787244734


### Run parameterized circuits

Use Estimator to run three experiments in a single job, leveraging parameter values to increase circuit reusability.

In [3]:
import numpy as np

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Step 1: Map classical inputs to a quantum problem
theta = Parameter("θ")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)

number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
individual_phases = [[ph] for ph in phases]

ZZ = SparsePauliOp.from_list([("ZZ", 1)])
ZX = SparsePauliOp.from_list([("ZX", 1)])
XZ = SparsePauliOp.from_list([("XZ", 1)])
XX = SparsePauliOp.from_list([("XX", 1)])
ops = [ZZ, ZX, XZ, XX]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
chsh_isa_circuit = pm.run(chsh_circuit)
isa_observables = [
    operator.apply_layout(chsh_isa_circuit.layout) for operator in ops
]

# Step 3: Execute using Qiskit primitives.

# Reshape observable array for broadcasting
reshaped_ops = np.fromiter(isa_observables, dtype=object)
reshaped_ops = reshaped_ops.reshape((4, 1))

estimator = Estimator(backend, options={"default_shots": int(1e4)})
job = estimator.run([(chsh_isa_circuit, reshaped_ops, individual_phases)])
# Get results for the first (and only) PUB
pub_result = job.result()[0]
print(f">>> Expectation values: {pub_result.data.evs}")
print(f">>> Standard errors: {pub_result.data.stds}")
print(f">>> Metadata: {pub_result.metadata}")

>>> Expectation values: [[ 9.92596584e-01  9.35114259e-01  7.94487856e-01  5.66406200e-01
   2.66882225e-01 -3.32576312e-02 -3.35450428e-01 -5.85703838e-01
  -8.13990787e-01 -9.52974839e-01 -9.93212466e-01 -9.37783081e-01
  -7.93461385e-01 -5.70717374e-01 -2.77146926e-01  2.21717541e-02
   3.23543375e-01  5.99458537e-01  8.15222552e-01  9.44968372e-01
   9.91364820e-01]
 [ 1.88870498e-02  3.22722199e-01  6.06849122e-01  8.18917844e-01
   9.54822485e-01  9.89106586e-01  9.40862491e-01  7.89355505e-01
   5.68048552e-01  2.77146926e-01 -2.09399900e-02 -3.19232200e-01
  -5.93915598e-01 -8.13169611e-01 -9.51127192e-01 -9.94033643e-01
  -9.34293083e-01 -7.80527862e-01 -5.62710908e-01 -2.91517508e-01
   2.29929302e-02]
 [-8.21176078e-04 -3.11020440e-01 -5.87962072e-01 -8.12348435e-01
  -9.42299550e-01 -9.96702465e-01 -9.45378960e-01 -8.02083734e-01
  -5.75644431e-01 -2.94186330e-01 -8.21176078e-04  3.11431028e-01
   5.82008545e-01  8.06600203e-01  9.54206603e-01  9.91364820e-01
   9.46816018e

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.

<CloudContent>
<Admonition type="caution">
The following code block will return an error for Open Plan users because it uses sessions. Open Plan workloads can run only in [job mode](/guides/execution-modes#job-mode) or [batch mode](/guides/execution-modes#batch-mode).
</Admonition>
</CloudContent>

In [4]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Session,
    EstimatorV2 as Estimator,
)

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = iqp(mat)
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = iqp(mat)
observable = SparsePauliOp("X" * 50)
another_observable = SparsePauliOp("Y" * 50)

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
another_isa_observable = another_observable.apply_layout(
    another_isa_circuit.layout
)

with Session(backend=backend) as session:
    estimator = Estimator(mode=session)

    estimator.options.resilience_level = 1

    job = estimator.run([(isa_circuit, isa_observable)])
    another_job = estimator.run(
        [(another_isa_circuit, another_isa_observable)]
    )
    result = job.result()
    another_result = another_job.result()

    # first job
    print(f" > Expectation value: {result[0].data.evs}")
    print(f" > Metadata: {result[0].metadata}")

    # second job
    print(f" > Another Expectation value: {another_result[0].data.evs}")
    print(f" > More Metadata: {another_result[0].metadata}")

 > Expectation value: 0.6953125
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}
 > Another Expectation value: 0.0211864406779661
 > More Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


<span id="sampler-examples"></span>
## Sampler examples

Generate entire error-mitigated quasi-probability distributions sampled from quantum circuit outputs. Leverage Sampler’s capabilities for search and classification algorithms like Grover’s and QVSM.

### Run a single experiment

Use Sampler to return the measurement outcome as bitstrings or counts of a single circuit.

In [5]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

sampler = Sampler(backend)
job = sampler.run([isa_circuit])
result = job.result()

# Get results for the first (and only) PUB
pub_result = result[0]

print(f" > First ten results: {pub_result.data.meas.get_bitstrings()[:10]}")

 > First ten results: ['1001011110001101110100010101110001010010111110000111001110110000010100000100000100000000001100101010111010001011011010010000001', '0010011010010011111100100110111100001110001001101001010011000100010001001010010010100000100000100011000111110110011000001001011', '0000100111100110001100111100000111010100100101001111001110101101110011011011100110000000100100001100111010001000010001000001000', '1000011010001000111000000100110101001001001100111110110011000010100011101000110001000011001101000011001111000010000000100000011', '0100010111000101010000101000010100110010101001011001000010001000110000010000000011000010001100001101100101011011000000101110001', '1111111110011100110100110000000000101100000010000111010011000000000001110001100001100100000100101010011010000110000011100000011', '1111111010001011100000100010000010011000100010100000100100100110101011101011000001101101100100000001110011101010001010100010000', '00100011010111101010100101000010000010011110110000000001001

### Run multiple experiments in a single job

Use Sampler to return the measurement outcome as bitstrings or counts of multiple circuits in one job.

In [6]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]
circuits = [iqp(mat) for mat in mats]
for circuit in circuits:
    circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuits = pm.run(circuits)

sampler = Sampler(mode=backend)
job = sampler.run(isa_circuits)
result = job.result()

for idx, pub_result in enumerate(result):
    print(
        f" > First ten results for pub {idx}: {pub_result.data.meas.get_bitstrings()[:10]}"
    )

 > First ten results for pub 0: ['0001111010010000100010010011111111000011000101000101101101000111010010101010101010110100010100011100101101101111100010110101100', '0100010010011010111010000010011001000000110011010111000000000001011010100100101001000010111101011011000100001001010111011000011', '1101011010010001000011011111000010010110001100110000111011010110100100001000011000101110001100001000100011101111001010100011010', '1010110010100100111111101001111011010000110000111000000000001111000100111100000011100000001101011000010011011101010111001110100', '1111011110100011100010001100010000010000011110100000010111000110000100101001001100100100000001001010000001010110000100100110011', '0110000001000011000110000000110001001010100000100111001110011100010100001011010010000000110000001100010101010110001011000010111', '1111011000110001110101000011010010000100100111010000011110111111011000011010001011100111110100010000001010110110010000010111010', '1010011101100101000001011111001001010000100011111

### Run parameterized circuits

Run several experiments in a single job, leveraging parameter values to increase circuit reusability.

In [7]:
import numpy as np
from qiskit.circuit.library import real_amplitudes
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

# Step 1: Map classical inputs to a quantum problem
circuit = real_amplitudes(num_qubits=n_qubits, reps=2)
circuit.measure_all()

# Define three sets of parameters for the circuit
rng = np.random.default_rng(1234)
parameter_values = [
    rng.uniform(-np.pi, np.pi, size=circuit.num_parameters) for _ in range(3)
]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

# Step 3: Execute using Qiskit primitives.
sampler = Sampler(backend)
job = sampler.run([(isa_circuit, parameter_values)])
result = job.result()
# Get results for the first (and only) PUB
pub_result = result[0]
# Get counts from the classical register "meas".
print(
    f" >> First ten results for the meas output register: {pub_result.data.meas.get_bitstrings()[:10]}"
)

 >> First ten results for the meas output register: ['0000100100011000101000001100101101011100000011110011011011001010000110101101100010001000010010110111001011111010101110101100001', '0100111100011100000110010100010011000110110111110011111011001001100110110001011101001101000011100001010011011000101101010010110', '1111010110101101101111010100110101010000111001111101001010111100111110101001110011111010010010101001010110110011110100011110001', '0101010100111001101010010100110001010101001111010011101000011001000110101110101001111110011001110111101011100001101110110000101', '1110001000100101001011010101011001010100100000001111101111101101000010001110111000101111110010000001010011000011001001010110101', '1111111001110101010010000100001101000001111001110011100000110011001011111100111011100011101000010101010000100111111110111110011', '0100111000110111111110011101011101110000100001001001001000011101011101001111111101111100001000001000001111011001111100010000101', '10011101100011110010100001100

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.

<CloudContent>
<Admonition type="caution">
The following code block will return an error for users on the Open Plan, because it uses sessions. Workloads on the Open Plan can run only in [job mode](/guides/execution-modes#job-mode) or [batch mode](/guides/execution-modes#batch-mode).
</Admonition>
</CloudContent>

In [8]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.quantum_info import random_hermitian
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = iqp(mat)
circuit.measure_all()
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = iqp(mat)
another_circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)

with Session(backend=backend) as session:
    sampler = Sampler(mode=session)
    job = sampler.run([isa_circuit])
    another_job = sampler.run([another_isa_circuit])
    result = job.result()
    another_result = another_job.result()

# first job

print(
    f" > The first ten measurement results of job 1: {result[0].data.meas.get_bitstrings()[:10]}"
)

 > The first ten measurement results of job 1: ['0000101101011011001000000100111000100001001100000010000110001011111101010111000100100101110011100110001000001011010100100000011', '1000001010101111111000000110000011111000000101010000000100100111001001001110010010110010001011101010000001100010100111101111101', '1011010111110000000111110111001100001100101000000011010000010101101110000011011001010010011000101001111110100100011100100000101', '0000111011001011110001000101010000101001101000000010010101111010100110101000101111001110000000101000000101110100110010001100101', '1001011001101000000100111010101011100001111101000000000111101101110100111100110101100001000100001101100010110010100110101000010', '0100010010001011000011110000101001101110101001000101110100000110101111000000110110101000010100000000101100010010110101100100010', '0010100111110110010101011110010111000000000101101000000011011011110111000000011100011100110000101000101011111100000010010100111', '0001010010100110010111000100010100

In [9]:
# second job
print(
    " > The first ten measurement results of job 2:",
    another_result[0].data.meas.get_bitstrings()[:10],
)

 > The first ten measurement results of job 2: ['1011000011100011001100010011100010001011000010110100110011011001001010110100000001001100010110100000001010010000000101000100011', '1011101111000010110001101010111010110101100000101100010101011101001110000000001001000001001101001100010011000010001100001111010', '1001101111100010011100101010001011011000010110110110111110010110011001010000010111000000100001000111110111000010000001100000001', '0011110000101001000000111001100101000111000110100010100011001111101100000010010010000000001011101110111001001110011000000101010', '1011100100001000010110100001110101101010001011011010001101001010100001110100010100100001111001010110111101000000100000111110010', '1100100011100111111110110101100000111011101000000100111001010111101110110000101011101111001001001010001101011010100011111110001', '1001110100100110111100010001110001000001011011111000010100001100100010000100000100010010101011111111100101001000000000110101010', '0011000111101111100001110000100010

## Next steps

<Admonition type="tip" title="Recommendations">
    - [Specify advanced runtime options.](runtime-options-overview)
    - Practice with primitives by working through the [Cost function lesson](https://learning.quantum.ibm.com/course/variational-algorithm-design/cost-functions#primitives) in IBM Quantum Learning.
    - Learn how to transpile locally in the [Transpile](./transpile/) section.
    - Try the [Submit pre-transpiled circuits](https://learning.quantum.ibm.com/tutorial/submitting-user-transpiled-circuits-using-primitives) tutorial.
    - Read [Migrate to V2 primitives](/migration-guides/v2-primitives).
    - Understand the [Job limits](/guides/job-limits) when sending a job to an IBM&reg; QPU.
</Admonition>